# General Imports

**Importing all libraries**

In [1]:
import numpy as np
import pandas as pd
import warnings

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance


Disable warnings

In [2]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

Generate train and test data

In [3]:
# Read Data
df = pd.read_csv("data_finish_prep.csv")

# Select only the usefull part of the df
df.drop(columns=['verschil_Lengte',
               'verschil_6 MWT', 'verschil_TUG',
               'verschil_Gewicht', 'verschil_Conditie',
               'verschil_Lenigheid', 'verschil_Knijpkracht'], inplace=True)

# Drop NaN values
df.dropna(inplace=True)

# Define X and y
X = df.drop(columns=['verschil_BMI'])
y = df['verschil_BMI']

# Split data into train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# MLR

**feature selection**

In [5]:
# Fit the Full Model
X =  sm.add_constant(X_train)  # Add constant term for the intercept
model = sm.OLS(y_train, X_train).fit()  # Fit the MLR model
best_model = model  # Initialize the best model
selected_features = X_train.columns.tolist()
best_features = X_train.columns.tolist()
best_features_rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

# Iteratively remove one feature at a time based on p-values
while True:
    # Compute p-values
    p_values = model.pvalues[1:]  # Exclude the constant term

    # Identify Insignificant Features
    insignificant_feature = p_values.idxmax()
    max_p_value = p_values.max()

    # Remove Insignificant Feature
    X = X.drop(insignificant_feature, axis=1)
    selected_features.remove(insignificant_feature)

    # When there a no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    model = sm.OLS(y_train, X).fit()
    
    # Evaluate Model Performance
    rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error').mean())

    # Check if the model performs better without the insignificant features
    if rmse <= best_features_rmse:
        best_features_rmse = rmse
        best_features = selected_features.copy()
        best_model = model

# Save results
best_model_mlr = LinearRegression().fit(X_train[best_features], y_train)
best_features_mlr = best_features

# Show results
print("Best features:", best_features)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features)))

Best features: ['SterfteRelatief', 'AfstandTotApotheek', 'AfstandTotSportterrein', 'AfstandTotBelangrijkOverstapstation', 'AfstandTotSauna', 'AfstandTotAttractie', 'EenOfMeerLangdurigeAandoeningen']
Dropped features: ['OppervlakteWater', 'AfstandTotPodiumkunstenTotaal', 'AfstandTotBioscoop', 'PercentageOnbewoond', 'AfstandTotKunstijsbaan', 'GemiddeldeWoningwaarde', 'Ondergewicht', 'AfstandTotBrandweerkazerne', 'InBezitOverigeVerhuurders', 'GeweldsEnSeksueleMisdrijven', 'ALandbouwBosbouwEnVisserij', 'MatigOvergewicht', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotOpenDroogNatTerrein', 'AfstandTotSchool', 'AfstandTotHuisartsenpraktijk', 'AfstandTotOpenNatNatuurlijkTerrein', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotTreinstationsTotaal', 'kPersonenMetLaagsteInkomen', 'PersonenautoSOverigeBrandstof', 'AfstandTotHotelED', 'PersonenautoSNaarOppervlakte', 'VoldoetAanBeweegrichtlijn', 'AfstandTotOvDagelLevensmiddelen', 'AfstandTotRestaurant', 'BouwjaarVanaf2000', 'AfstandTotWarenhuis',

*Er zijn geen hyperparameters om te optimaliseren.*

# Support Vector Machines

**Hyperparameter optimalisatie**

In [6]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    classifier = model(**params)
    
    # Calculating accuracy based on cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    return scores.mean()

params_ranges = {
    'C': (0.1, 10),
    'kernel_int': (1, 4),
    'gamma': (0.001, 0.1)
}

kernel_mapping = {
    1: 'linear',
    2: 'rbf',
    3: 'poly',
    4: 'sigmoid'
}

# Example usage with SVM
model = SVR
model_bo = BayesianOptimization(f=lambda C, kernel_int, gamma:
                                    bo_params_generic(model, {
                                        'C': C,
                                        'kernel': kernel_mapping[int(kernel_int)],
                                        'gamma': gamma
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_svm = model(
    C=params['C'],
    kernel=kernel_mapping[int(params['kernel_int'])],
    gamma=params['gamma']
)

# Fit the model
best_model_svm.fit(X_train, y_train)


|   iter    |  target   |     C     |   gamma   | kernel... |
-------------------------------------------------------------
| 1         | -1.461    | 0.6336    | 0.05196   | 1.424     |
| 2         | -1.462    | 1.145     | 0.05214   | 3.62      |
| 3         | -1.46     | 7.784     | 0.05134   | 2.684     |
| 4         | -1.465    | 7.644     | 0.09708   | 1.771     |
| 5         | -1.463    | 4.671     | 0.07817   | 1.495     |
| 6         | -1.462    | 3.835     | 0.01504   | 2.826     |
| 7         | -1.458    | 1.859     | 0.08366   | 3.973     |
| 8         | -1.46     | 8.995     | 0.04196   | 2.89      |
| 9         | -1.46     | 9.289     | 0.0327    | 3.033     |
| 10        | -1.463    | 2.138     | 0.01165   | 3.4       |
| 11        | -1.46     | 2.929     | 0.00238   | 3.148     |
| 12        | -1.459    | 9.652     | 0.04476   | 3.092     |
| 13        | -1.459    | 1.888     | 0.06154   | 3.99      |
| 14        | -1.459    | 2.77      | 0.03955   | 2.914     |
| 15    

SVR(C=9.46640201332018, gamma=0.08635486453021478, kernel='poly')

**Feature Selection**

In [7]:
# Get the accuracy score of the model with all features
best_accuracy = cross_val_score(best_model_svm, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()
best_model = best_model_svm
selected_features = X_train.columns
best_features = selected_features

while len(selected_features) > 1:
    try:
        # Retrieve the coefficients from the SVM model
        coefficients = best_model_svm.coef_[0]

        # Sort features based on their coefficient magnitudes
        sorted_indices = np.argsort(np.abs(coefficients))
        selected_features = [X_train.columns[i] for i in sorted_indices[::-1]]

        # Remove the least important feature
        selected_features = selected_features[:-1]
        
        # Refit the linear SVM classifier
        best_model_svm_classification.fit(X_train[selected_features], y_train)
        
        # Evaluate Model Performance
        accuracy = cross_val_score(best_model_svm, X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

        # Check if the model performs better without the insignificant features
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.copy()
            best_model = best_model_svm
    except:
        print("Scince the kernal is not linear, this feature selection is not possible") 
        break

# Save results
best_model_svm = best_model
best_features_svm = best_features

# Show results
print("Best features:", best_features_svm)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_svm)))

Scince the kernal is not linear, this feature selection is not possible
Best features: Index(['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief',
       'GemiddeldeWoningwaarde', 'PercentageOnbewoond',
       'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
       'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen',
       'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed',
       'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
       'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk',
       'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater',
       'TotaalDiefstalUitWoningSchuurED',
       'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven',
       'AfstandTotHuisartsenpost', 'AfstandTotApotheek',
       'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen',
       'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED',
       'AfstandTotResta

# Random Forest

**Feature selection**

In [8]:
# Get the accuracy score of the model with all the features
rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
best_rf = rf
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Make DataFrame of feature impotances
    result = permutation_importance(
            rf, X_train[selected_features], y_train, n_repeats=10, random_state=42, n_jobs=-1
        )
    df_importances = pd.DataFrame({'feature': X_train[selected_features].columns, 'importance': result.importances_mean, 'std': result.importances_std})
    df_importances = df_importances.sort_values('importance')

    # Remove least important feature
    selected_features.remove(df_importances['feature'].iloc[0])

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = rf

# Save results
best_model_rf = best_model
best_features_rf = best_features

# Show results
print("Best features:", best_features_rf)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_rf)))

Best features: ['AfstandTotZonnebank']
Dropped features: ['OppervlakteWater', 'AfstandTotPodiumkunstenTotaal', 'AfstandTotBioscoop', 'PercentageOnbewoond', 'AfstandTotKunstijsbaan', 'GemiddeldeWoningwaarde', 'Ondergewicht', 'AfstandTotBrandweerkazerne', 'InBezitOverigeVerhuurders', 'GeweldsEnSeksueleMisdrijven', 'ALandbouwBosbouwEnVisserij', 'MatigOvergewicht', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotSauna', 'AfstandTotOpenDroogNatTerrein', 'AfstandTotSchool', 'AfstandTotHuisartsenpraktijk', 'AfstandTotOpenNatNatuurlijkTerrein', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotTreinstationsTotaal', 'kPersonenMetLaagsteInkomen', 'PersonenautoSOverigeBrandstof', 'AfstandTotHotelED', 'PersonenautoSNaarOppervlakte', 'VoldoetAanBeweegrichtlijn', 'AfstandTotOvDagelLevensmiddelen', 'AfstandTotRestaurant', 'BouwjaarVanaf2000', 'AfstandTotWarenhuis', 'TotaalDiefstalUitWoningSchuurED', 'AfstandTotVerblijfsrecreatiefTerrein', 'Mantelzorger', 'Rokers', 'ZwaarBelasteMantelzorgers', 'AfstandTo

**Hyperparameter optimalisatie**

In [9]:
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestRegressor(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    return cross_val_score(clf, X_train[best_features_rf], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

# Create Bayesian Optimization
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

# Bayesian optimization
results = rf_bo.maximize(n_iter=40, init_points=10)

# Getting best hyperparameters from bayesian optimization
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

# Making random forest model with the best hyperparameters
# criterion entropy is after the hyperparameter optimization since it's more accurate but increases training time
best_model_rf_classification = RandomForestRegressor(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

best_model_rf_classification.fit(X_train[best_features_rf], y_train)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------


| 1         | -1.431    | 14.59     | 0.7003    | 8.114     | 889.7     |
| 2         | -1.442    | 2.336     | 0.8459    | 2.45      | 485.4     |
| 3         | -1.431    | 7.726     | 0.3436    | 3.167     | 241.2     |
| 4         | -1.431    | 8.023     | 0.928     | 8.678     | 318.8     |
| 5         | -1.442    | 1.773     | 0.8244    | 5.41      | 658.4     |
| 6         | -1.433    | 5.467     | 0.8026    | 6.991     | 798.1     |
| 7         | -1.436    | 1.03      | 0.9623    | 10.91     | 109.2     |
| 8         | -1.431    | 6.815     | 0.793     | 2.236     | 827.2     |
| 9         | -1.431    | 13.36     | 0.8899    | 9.789     | 642.8     |
| 10        | -1.431    | 7.929     | 0.5725    | 10.7      | 644.6     |
| 11        | -1.433    | 5.117     | 0.3451    | 11.75     | 633.5     |
| 12        | -1.431    | 12.66     | 0.5007    | 6.587     | 815.3     |
| 13        | -1.431    | 18.99     | 0.9924    | 8.779     | 829.6     |
| 14        | -1.431    | 5.872     | 

RandomForestRegressor(bootstrap=False, max_depth=15,
                      max_features=0.7003310088823661, min_samples_split=8,
                      n_estimators=890)

# neural network

**Feature selection**

In [10]:
# Get the accuracy score of the model with all the features
mlp = MLPRegressor(random_state=0).fit(X_train, y_train)
best_model = mlp
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(MLPRegressor(random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Retrieve the learned weights
    weights = mlp.coefs_[0]  # Weights connecting input features to the first hidden layer

    # Normalize the weights
    normalized_weights = np.abs(weights) / np.sum(np.abs(weights), axis=0)

    # Calculate feature importance
    feature_importance = np.mean(normalized_weights, axis=1)

    # Sort features based on their importance
    sorted_indices = np.argsort(feature_importance)
    selected_features = [selected_features[i] for i in sorted_indices[:-1]]

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    mlp = MLPRegressor(random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(MLPRegressor(random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = mlp

# Save results
best_model_nn = best_model
best_features_nn = best_features

# Show results
print("Best features:", best_features_nn)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_nn)))

Best features: ['ErnstigOvergewichtObesitas', 'UrenMantelzorgPerWeek', 'ZwaarBelasteMantelzorgers', 'AfstandTotSchool', 'AfstandTotBos', 'kPersonenMetLaagsteInkomen', 'VoldoetAanBeweegrichtlijn', 'OvermatigDrinker', 'InBezitWoningcorporatie', 'MatigOvergewicht', 'PersonenautoSPerHuishouden', 'AfstandTotWarenhuis', 'ErvarenGezondheidGoedZeerGoed', 'BouwjaarVanaf2000', 'AfstandTotConsultatiebureau', 'NederlandseAntillenEnAruba', 'AfstandTotBibliotheek', 'BeperkingInZien', 'AfstandTotHotelED', 'OppervlakteWater', 'PersonenautoSNaarOppervlakte', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotBrandweerkazerne', 'Ondergewicht', 'GeboorteRelatief', 'AfstandTotTreinstationsTotaal', 'AfstandTotCafeED', 'GeweldsEnSeksueleMisdrijven', 'AfstandTotSauna', 'AfstandTotBegraafplaats', 'AfstandTotZwembad', 'AfstandTotPodiumkunstenTotaal', 'KLFinancieleDienstenOnroerendGoed', 'AfstandTotOpenNatNatuurlijkTerrein', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotPoppodium', 'AfstandTotBuitenschoolseOpv

**Hyperparameter optimalisatie**

In [11]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Calculating rmse based on
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return scores.mean()

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000)
}

# Example usage with Neural Network
model = MLPRegressor
model_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train[best_features_nn], y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_nn = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_nn.fit(X_train[best_features_nn], y_train)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | -1.192    | 0.05145   | 97.22     | 0.02344   | 919.3     |
| 2         | -1.192    | 0.07949   | 95.44     | 0.08917   | 458.7     |
| 3         | -1.195    | 0.02829   | 91.05     | 0.05618   | 918.9     |
| 4         | -1.195    | 0.07181   | 58.68     | 0.09417   | 412.2     |
| 5         | -1.194    | 0.01843   | 99.69     | 0.02227   | 595.3     |
| 6         | -1.194    | 0.08451   | 85.88     | 0.02077   | 386.4     |
| 7         | -1.191    | 0.00817   | 14.62     | 0.07069   | 628.4     |
| 8         | -1.19     | 0.04943   | 18.1      | 0.0867    | 844.1     |
| 9         | -1.194    | 0.03918   | 42.7      | 0.05326   | 744.4     |
| 10        | -1.195    | 0.05927   | 76.74     | 0.01524   | 713.4     |
| 11        | -1.197    | 0.06102   | 49.27     | 0.01517   | 819.5     |
| 12        | -1.195    | 0.0786    | 

MLPRegressor(alpha=0.07908331240409669, hidden_layer_sizes=(96,),
             learning_rate_init=0.06985298045720616, max_iter=726)

# plotting the bar chart

**using train dataset**

In [12]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_val = []

for model in regression_models:
    try:
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=10)
        y_true = y_train
    except: # cv can't be higher then n_splits in the dt
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=5)
        y_true = y_train

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_val.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_val

[{'model': 'Multiple Linear Regression',
  'mae': 0.8671939412421841,
  'mse': 1.4268453165906185,
  'rmse': 1.1945063066349288,
  'r2': 0.004033710129095258,
  'adj_r2': -0.0736894117114435},
 {'model': 'Random Forest',
  'mae': 0.8660175252217047,
  'mse': 1.4303383451797227,
  'rmse': 1.195967535169631,
  'r2': 0.0015955069939328492,
  'adj_r2': -0.07631788710910459},
 {'model': 'Neural Network',
  'mae': 0.8778584647650217,
  'mse': 1.4437759688925358,
  'rmse': 1.2015722903315205,
  'r2': -0.007784220491812377,
  'adj_r2': -0.08642958886910002},
 {'model': 'Support Vector Machine',
  'mae': 0.8606249650525079,
  'mse': 1.455202466589116,
  'rmse': 1.2063177303634047,
  'r2': -0.01576014218756705,
  'adj_r2': -0.09502793477745519}]

In [13]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_val]
metric_labels = list(regression_stats_val[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_val] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

**Evaluating on test data**

In [14]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_test = []

for model in regression_models:
    # Make predictions on the test set
    y_pred = model['model'].predict(X_test[model['features']])

    # Defining Actual values
    y_true = y_test.copy()

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_test.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_test

[{'model': 'Multiple Linear Regression',
  'mae': 0.9045298004108571,
  'mse': 1.5779823658292136,
  'rmse': 1.2561776808354834,
  'r2': 0.0033979071955164697,
  'adj_r2': -0.40456786178877513},
 {'model': 'Random Forest',
  'mae': 0.8941572116607733,
  'mse': 1.5706296636202557,
  'rmse': 1.2532476465648184,
  'r2': 0.00804163361977539,
  'adj_r2': -0.3980231947230066},
 {'model': 'Neural Network',
  'mae': 0.8977895369858052,
  'mse': 1.5946822122768582,
  'rmse': 1.2628072743997234,
  'r2': -0.007149170059361776,
  'adj_r2': -0.41943245604857426},
 {'model': 'Support Vector Machine',
  'mae': 0.8924119168242366,
  'mse': 1.6334579766730493,
  'rmse': 1.2780680641785278,
  'r2': -0.031638675635699665,
  'adj_r2': -0.45394690542809135}]

In [15]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_test]
metric_labels = list(regression_stats_test[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_test] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

Naive bayes

In [16]:
# Defining Actual values
y_true = y_test.copy()

# Defining Predicted values
y_pred = np.full_like(y_test, y_train.mean())

# Mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

# Root Mean Squared Error
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R2 Score
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

# Adjusted R2 Score
n = len(y_true)  # Number of samples
p = X_test.shape[1]  # Number of predictors (features) in X
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
print("Adjusted R2 Score:", adj_r2)

Mean Absolute Error: 0.9067514130956161
Mean Squared Error: 1.5936473903413322
Root Mean Squared Error: 1.2623974771605544
R2 Score: -0.006495610343513203
Adjusted R2 Score: -0.41851135726775834


Save best model as Pickle

In [17]:
import pickle

# Save model
file_path = 'Saved Models/BMI.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_model_mlr, file)

# Save features
with open('Saved Models/best_features_BMI.txt', 'w') as file:
    file.write(f'{best_features_mlr}')